# Deep Learning Models

In [1]:
import pandas as pd

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn import metrics
from sklearn.metrics import accuracy_score

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, GRU
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.sequence import TimeseriesGenerator
from tensorflow.keras.wrappers.scikit_learn import KerasRegressor

import matplotlib.pyplot as plt

In [2]:
Final_df = pd.read_csv('./Clean_dataframe/clean_df.csv')

In [3]:
features = ['LONGITUDE', 'ELEVATION','TMIN', 'TMAX' ,'PRCP','TAVG']
X = Final_df[features]
y = Final_df['sea_level']

X_train, X_test, y_train, y_test = train_test_split(X,y,random_state=42)

In [4]:
sc = StandardScaler()
X_train_sc = sc.fit_transform(X_train)
X_test_sc = sc.transform(X_test)

In [5]:
# construct network
model = Sequential()

# first layer
model.add(Dense(32, input_shape=(6,), activation='relu'))

# second layer
model.add(Dense(32, activation='relu'))

# output layer
model.add(Dense(1))

In [6]:
model.compile(loss='mse', optimizer='adam')

In [7]:
model.fit(X_train_sc, y_train, validation_data=(X_test_sc, y_test), epochs=10, batch_size=512);

Epoch 1/10
139/139 [==============================] - 1s 2ms/step - loss: 7942597.5000 - val_loss: 7861225.5000
Epoch 2/10
139/139 [==============================] - 0s 1ms/step - loss: 7493249.0000 - val_loss: 6956675.5000
Epoch 3/10
139/139 [==============================] - 0s 1ms/step - loss: 6029301.5000 - val_loss: 4968781.5000
Epoch 4/10
139/139 [==============================] - 0s 1ms/step - loss: 3886355.7500 - val_loss: 2916128.0000
Epoch 5/10
139/139 [==============================] - 0s 1ms/step - loss: 2274165.2500 - val_loss: 1814074.6250
Epoch 6/10
139/139 [==============================] - 0s 1ms/step - loss: 1529334.7500 - val_loss: 1342939.8750
Epoch 7/10
139/139 [==============================] - 0s 1ms/step - loss: 1163916.8750 - val_loss: 1049986.5000
Epoch 8/10
139/139 [==============================] - 0s 1ms/step - loss: 918434.8125 - val_loss: 842158.7500
Epoch 9/10
139/139 [==============================] - 0s 927us/step - loss: 750227.1875 - val_loss: 702041

In [8]:
model.predict(X_test_sc)

array([[2163.4307],
       [2762.0334],
       [1680.4481],
       ...,
       [1131.8014],
       [1909.7744],
       [2792.4814]], dtype=float32)

In [9]:
model.evaluate(X_test_sc, y_test, batch_size=512)

47/47 [==============================] - 0s 665us/step - loss: 610055.9375


610055.9375

## GridSearch

In [10]:
def model_fn_adv(hidden_neurons = 32, hidden_layers = 5, dropout = 0.5):
    # build framework of model using for loop and if statement
    model = Sequential()
    
    # adding number of layers = specified in calling of function
    for layer in range(hidden_layers):
        if layer == 0:
            model.add(Dense(hidden_neurons, activation = 'relu', input_shape = (6,)))
            model.add(Dropout(dropout))
        else:
            model.add(Dense(hidden_neurons, activation = 'relu'))
            model.add(Dropout(dropout))
    
    # out put layer added here, Identity link function used as 'None'
    model.add(Dense(1, activation = None))

    # using mse as the metric to solve for 
    model.compile(loss = 'mse', optimizer = 'adam')
    
    return model

In [11]:
nn = KerasRegressor(build_fn = model_fn_adv, batch_size = 512, verbose = 0)

In [12]:
nn_params_deep = {
    'hidden_neurons' : [20, 32, 64],
    'hidden_layers'  : [2, 3, 5],
    'dropout'        : [0.1, 0.2, 0.3, 0.4, 0.5],
    'epochs'         : [10, 20, 30, 40, 50]
}

In [13]:
gs = GridSearchCV(nn, param_grid = nn_params_deep, cv = 5)

In [14]:
gs.fit(X_train_sc, y_train)

GridSearchCV(cv=5,
             estimator=<tensorflow.python.keras.wrappers.scikit_learn.KerasRegressor object at 0x7fc47c236b20>,
             param_grid={'dropout': [0.1, 0.2, 0.3, 0.4, 0.5],
                         'epochs': [10, 20, 30, 40, 50],
                         'hidden_layers': [2, 3, 5],
                         'hidden_neurons': [20, 32, 64]})

In [15]:
gs.best_params_

{'dropout': 0.1, 'epochs': 10, 'hidden_layers': 5, 'hidden_neurons': 64}

In [16]:
gs.best_score_

-43316.03046875

In [17]:
# Make predictions 
preds = gs.predict(X_test_sc)

In [18]:
# Check the R-squared
metrics.r2_score(y_test, preds)

0.9608463849025666

## Recurrent Neural Networks

In [27]:
features = ['LONGITUDE', 'ELEVATION','TMIN', 'TMAX' ,'PRCP','TAVG']
X = Final_df[features]
y = Final_df['sea_level'].values

X_train, X_test, y_train, y_test = train_test_split(X,y,random_state=42, shuffle = False)

In [28]:
ss = StandardScaler()
X_train_sc = ss.fit_transform(X_train)
X_test_sc = ss.transform(X_test)

In [29]:
train_sequences = TimeseriesGenerator(X_train_sc, y_train, length=3, batch_size=64)

In [30]:
test_sequences = TimeseriesGenerator(X_test_sc, y_test, length=3, batch_size=64)

In [31]:
input_shape = train_sequences[0][0][0].shape

In [32]:
model = Sequential()
model.add(GRU(8, input_shape=input_shape, return_sequences=True)) # True if next layer is RNN
model.add(GRU(8, return_sequences=False)) # False if next layer is Dense

model.add(Dense(4, activation='relu'))

model.add(Dense(1))

In [33]:
model.compile(optimizer='adam', loss='mse')

history = model.fit(train_sequences, validation_data=test_sequences, epochs=50, verbose=0)

In [35]:
# Add early stopping
from tensorflow.keras.callbacks import EarlyStopping

model2 = Sequential()
model2.add(GRU(8, input_shape=input_shape, return_sequences=True)) # True if next layer is RNN
model2.add(GRU(8, return_sequences=False)) # False if next layer is Dense

model2.add(Dense(4, activation='relu'))

model2.add(Dense(1))

model2.compile(optimizer='adam', loss='mse')

es = EarlyStopping(patience = 5)

history2 = model2.fit(train_sequences, validation_data=test_sequences, epochs=100, batch_size = 64, callbacks = [es])

Epoch 1/100
1110/1110 [==============================] - 6s 3ms/step - loss: 8084719.0000 - val_loss: 6919259.0000
Epoch 2/100
1110/1110 [==============================] - 3s 3ms/step - loss: 7515687.0000 - val_loss: 6231496.0000
Epoch 3/100
1110/1110 [==============================] - 3s 3ms/step - loss: 6641884.0000 - val_loss: 5338479.0000
Epoch 4/100
1110/1110 [==============================] - 3s 3ms/step - loss: 5606938.0000 - val_loss: 4363603.0000
Epoch 5/100
1110/1110 [==============================] - 3s 3ms/step - loss: 4522293.0000 - val_loss: 3402316.5000
Epoch 6/100
1110/1110 [==============================] - 3s 3ms/step - loss: 3482965.5000 - val_loss: 2541319.2500
Epoch 7/100
1110/1110 [==============================] - 3s 3ms/step - loss: 2571021.0000 - val_loss: 1850052.6250
Epoch 8/100
1110/1110 [==============================] - 3s 3ms/step - loss: 1851107.8750 - val_loss: 1375370.1250
Epoch 9/100
1110/1110 [==============================] - 3s 3ms/step - loss: 136